In [2]:
import pandas as pd
import os

# Base path
base_path = "C:/Users/vongu/OneDrive/Desktop/finance/finance_projet_sentiment_analyse"

# Relative paths
relative_paths = [
    "new_data/risk_free_rate_added/BHP_Group_add_risk_free_rate.csv",
    "new_data/risk_free_rate_added/BP_PLC_add_risk_free_rate.csv",
    "new_data/risk_free_rate_added/FMC_Corp_add_risk_free_rate.csv",
    "new_data/risk_free_rate_added/Stora_Enso_add_risk_free_rate.csv",
    "new_data/risk_free_rate_added/Total_Energies_add_risk_free_rate.csv"
]

# Combine base path with relative paths
file_paths = [os.path.join(base_path, relative_path) for relative_path in relative_paths]

stock_names = ["BHP_Group", "BP_PLC", "FMC_Corp", "Stora_Enso", "Total_Energies"]

# Read the data into a dictionary of DataFrames
stock_data = {name: pd.read_csv(file_path) for name, file_path in zip(stock_names, file_paths)}

# Extract 'Put-Call Ratio' data for all stocks and align with dates
put_call_ratios = {name: df[['Date', 'Put-Call Ratio']] for name, df in stock_data.items()}

# Merge all stock put-call ratios into a single DataFrame
merged_data = pd.DataFrame()
for name, df in put_call_ratios.items():
    df = df.rename(columns={'Put-Call Ratio': name})  
    if merged_data.empty:
        merged_data = df
    else:
        merged_data = pd.merge(merged_data, df, on='Date', how='outer')

# Ensure Date is in datetime format and set as index
merged_data['Date'] = pd.to_datetime(merged_data['Date'])
merged_data.set_index('Date', inplace=True)

In [2]:
merged_data.head()

,BHP_Group,BP_PLC,FMC_Corp,Stora_Enso,Total_Energies
Date,,,,,
2023-01-02,NaN,NaN,NaN,NaN,0.00
2023-01-03,0.00,0.43,0.00,0.43,0.43
2023-01-04,1.12,3.05,1.12,3.05,3.05
2023-01-05,0.00,0.77,0.00,0.77,0.77
2023-01-06,0.99,0.80,0.99,0.80,0.80


In [3]:
bullish_threshold = 0.7
bearish_threshold = 1.2

# Define function to calculate signals based on Put-Call Ratio
def calculate_signals_pcr(put_call_ratios, bullish_threshold, bearish_threshold):
    """
    Define function to calculate signals based on Put-Call Ratio, with streak reset for trend changes.
    """
    streak = 0
    signals = []
    previous_signal = None

    for ratio in put_call_ratios:
        if ratio < bullish_threshold:  # Bullish
            if previous_signal != "Bullish":
                streak = 0  # Reset streak on trend change
            streak += 1
            current_signal = "Bullish"
        elif ratio > bearish_threshold:  # Bearish
            if previous_signal != "Bearish":
                streak = 0  # Reset streak on trend change
            streak -= 1
            current_signal = "Bearish"
        else:  # Neutral
            if previous_signal != "Neutral":
                streak = 0  # Reset streak on trend change
            current_signal = "Neutral"

        # Assign Buy, Sell, or Hold based on streak
        if streak >= 3:
            signals.append("Buy")
        elif streak <= -3:
            signals.append("Sell")
        else:
            signals.append("Hold")

        # Update previous signal
        previous_signal = current_signal

    return signals

# Generate buy/sell/hold signals for each stock
signal_data = merged_data.copy()
for stock in stock_names:
    signal_data[f"{stock}_Signal"] = calculate_signals_pcr(signal_data[stock], bullish_threshold, bearish_threshold)


In [4]:
signal_data.head(20)

,BHP_Group,BP_PLC,FMC_Corp,Stora_Enso,Total_Energies,BHP_Group_Signal,BP_PLC_Signal,FMC_Corp_Signal,Stora_Enso_Signal,Total_Energies_Signal
Date,,,,,,,,,,
2019-01-03,NaN,1.59,NaN,1.59,1.59,Hold,Hold,Hold,Hold,Hold
2019-01-04,NaN,3.08,NaN,3.08,3.08,Hold,Hold,Hold,Hold,Hold
2019-01-07,NaN,1.14,NaN,1.14,1.14,Hold,Hold,Hold,Hold,Hold
2019-01-08,NaN,1.72,NaN,1.72,1.72,Hold,Hold,Hold,Hold,Hold
2019-01-09,NaN,1.39,NaN,1.39,1.39,Hold,Hold,Hold,Hold,Hold
2019-01-10,NaN,2.28,NaN,2.28,2.28,Hold,Sell,Hold,Sell,Sell
2019-01-11,NaN,1.23,NaN,1.23,1.23,Hold,Sell,Hold,Sell,Sell
2019-01-14,NaN,2.88,NaN,2.88,2.88,Hold,Sell,Hold,Sell,Sell
2019-01-15,NaN,3.71,NaN,3.71,3.71,Hold,Sell,Hold,Sell,Sell


In [38]:
# Initialize portfolio weights
initial_weight = 1 / len(stock_names)
weights = {stock: initial_weight for stock in stock_names}

# Function to adjust weights based on signals
def adjust_weights(signal, current_weight):
    if signal == "Buy":
        return current_weight * 1.1  # Increase by 10%
    elif signal == "Sell":
        return current_weight * 0.9  # Decrease by 10%
    return current_weight  # Hold


# Calculate daily portfolio weights
weight_data = pd.DataFrame(index=signal_data.index)
for stock in stock_names:
    weight_data[f"{stock}_Weight"] = initial_weight
    for i, signal in enumerate(signal_data[f"{stock}_Signal"]):
        if i > 0:  # Adjust weights dynamically
            weight_data.iloc[i, weight_data.columns.get_loc(f"{stock}_Weight")] = adjust_weights(
                signal,
                weight_data.iloc[i - 1, weight_data.columns.get_loc(f"{stock}_Weight")]
            )

# Normalize weights to ensure they sum to 1 each day
weight_data["Total_Weight"] = weight_data.sum(axis=1)
for stock in stock_names:
    weight_data[f"{stock}_Normalized_Weight"] = weight_data[f"{stock}_Weight"] / weight_data["Total_Weight"]


In [39]:
weight_data.head(10)

,BHP_Group_Weight,BP_PLC_Weight,FMC_Corp_Weight,Stora_Enso_Weight,Total_Energies_Weight,Total_Weight,BHP_Group_Normalized_Weight,BP_PLC_Normalized_Weight,FMC_Corp_Normalized_Weight,Stora_Enso_Normalized_Weight,Total_Energies_Normalized_Weight
Date,,,,,,,,,,,
2023-01-02,0.2,0.200,0.2,0.200,0.200,1.000,0.200000,0.200000,0.200000,0.200000,0.200000
2023-01-03,0.2,0.200,0.2,0.200,0.200,1.000,0.200000,0.200000,0.200000,0.200000,0.200000
2023-01-04,0.2,0.200,0.2,0.200,0.200,1.000,0.200000,0.200000,0.200000,0.200000,0.200000
2023-01-05,0.2,0.200,0.2,0.200,0.200,1.000,0.200000,0.200000,0.200000,0.200000,0.200000
2023-01-06,0.2,0.200,0.2,0.200,0.200,1.000,0.200000,0.200000,0.200000,0.200000,0.200000
2023-01-09,0.2,0.200,0.2,0.200,0.200,1.000,0.200000,0.200000,0.200000,0.200000,0.200000
2023-01-10,0.2,0.200,0.2,0.200,0.200,1.000,0.200000,0.200000,0.200000,0.200000,0.200000
2023-01-11,0.2,0.180,0.2,0.180,0.180,0.940,0.212766,0.191489,0.212766,0.191489,0.191489
2023-01-12,0.2,0.162,0.2,0.162,0.162,0.886,0.225734,0.182844,0.225734,0.182844,0.182844


In [40]:
# Define periods for analysis
latest_date = signal_data.index.max()
periods = {
    "3 Months": latest_date - pd.DateOffset(months=3),
    "1 Year": latest_date - pd.DateOffset(years=1),
    "2 Years": latest_date - pd.DateOffset(years=2),
    "All Periods": signal_data.index.min(),
}

# Calculate average weights for each period
period_weights = {}
for period_name, start_date in periods.items():
    period_data = weight_data[weight_data.index >= start_date]
    avg_weights = period_data[[f"{stock}_Normalized_Weight" for stock in stock_names]].mean()
    period_weights[period_name] = avg_weights

# Convert period weights to a DataFrame
period_weights_df = pd.DataFrame(period_weights).T

In [41]:
period_weights_df.head()

,BHP_Group_Normalized_Weight,BP_PLC_Normalized_Weight,FMC_Corp_Normalized_Weight,Stora_Enso_Normalized_Weight,Total_Energies_Normalized_Weight
3 Months,0.470206,0.021311,0.470206,0.021311,0.016966
1 Year,0.388699,0.077546,0.388699,0.077546,0.067511
2 Years,0.388699,0.077546,0.388699,0.077546,0.067511
All Periods,0.388699,0.077546,0.388699,0.077546,0.067511


In [1]:
import pandas as pd
import os

def merge_put_call_ratios(file_paths, stock_names):
    """
    Merges 'Put-Call Ratio' data for multiple stocks into a single DataFrame aligned by date.
    """
    # Read the data into a dictionary of DataFrames
    stock_data = {name: pd.read_csv(file_path) for name, file_path in zip(stock_names, file_paths)}

    # Extract 'Put-Call Ratio' data for all stocks and align with dates
    put_call_ratios = {name: df[['Date', 'Put-Call Ratio']] for name, df in stock_data.items()}

    # Merge all stock put-call ratios into a single DataFrame
    merged_data = None
    for name, df in put_call_ratios.items():
        df = df.rename(columns={'Put-Call Ratio': name})  # Rename 'Put-Call Ratio' to the stock name
        if merged_data is None:
            merged_data = df  # Initialize with the first DataFrame
        else:
            merged_data = pd.merge(merged_data, df, on='Date', how='outer')

    # Ensure Date is in datetime format and set as index
    merged_data['Date'] = pd.to_datetime(merged_data['Date'])
    merged_data.set_index('Date', inplace=True)

    return merged_data

def calculate_signals_pcr(put_call_ratios, bullish_threshold, bearish_threshold):
    """
    Define function to calculate signals based on Put-Call Ratio, with streak reset for trend changes.
    """
    streak = 0
    signals = []
    previous_signal = None

    for ratio in put_call_ratios:
        if ratio < bullish_threshold:  # Bullish
            if previous_signal != "Bullish":
                streak = 0  # Reset streak on trend change
            streak += 1
            current_signal = "Bullish"
        elif ratio > bearish_threshold:  # Bearish
            if previous_signal != "Bearish":
                streak = 0  # Reset streak on trend change
            streak -= 1
            current_signal = "Bearish"
        else:  # Neutral
            if previous_signal != "Neutral":
                streak = 0  # Reset streak on trend change
            current_signal = "Neutral"

        # Assign Buy, Sell, or Hold based on streak
        if streak >= 3:
            signals.append("Buy")
        elif streak <= -3:
            signals.append("Sell")
        else:
            signals.append("Hold")

        # Update previous signal
        previous_signal = current_signal

    return signals

def calculate_dynamic_portfolio_weights(signal_data, stock_names):
    """
    Calculates daily portfolio weights dynamically based on signal data for multiple stocks.

    Parameters:
        signal_data (pd.DataFrame): DataFrame containing signal data for stocks. Columns should include '{stock}_Signal'.
        stock_names (list): List of stock names.

    Returns:
        pd.DataFrame: A DataFrame containing normalized portfolio weights for each stock.
    """
    # Initialize portfolio weights
    initial_weight = 1 / len(stock_names)

    # Prepare a DataFrame to store weight data
    weight_data = pd.DataFrame(index=signal_data.index)

    # Calculate dynamic weights for each stock
    for stock in stock_names:
        weight_data[f"{stock}_Weight"] = initial_weight  # Set initial weight
        for i, signal in enumerate(signal_data[f"{stock}_Signal"]):
            if i > 0:  # Adjust weights dynamically based on the signal
                # Adjust weights based on signals
                if signal == "Buy":
                    weight_data.iloc[i, weight_data.columns.get_loc(f"{stock}_Weight")] = (
                        weight_data.iloc[i - 1, weight_data.columns.get_loc(f"{stock}_Weight")] * 1.1
                    )  # Increase by 10%
                elif signal == "Sell":
                    weight_data.iloc[i, weight_data.columns.get_loc(f"{stock}_Weight")] = (
                        weight_data.iloc[i - 1, weight_data.columns.get_loc(f"{stock}_Weight")] * 0.9
                    )  # Decrease by 10%
                else:
                    weight_data.iloc[i, weight_data.columns.get_loc(f"{stock}_Weight")] = (
                        weight_data.iloc[i - 1, weight_data.columns.get_loc(f"{stock}_Weight")]
                    )  # Hold

    # Normalize weights to ensure they sum to 1 each day
    weight_data["Total_Weight"] = weight_data[[f"{stock}_Weight" for stock in stock_names]].sum(axis=1)
    for stock in stock_names:
        weight_data[f"{stock}_Normalized_Weight"] = (
            weight_data[f"{stock}_Weight"] / weight_data["Total_Weight"]
        )

    # Drop the intermediate 'Total_Weight' column
    weight_data.drop(columns=["Total_Weight"], inplace=True)

    return weight_data





file_paths = [
    "C:/Users/vongu/OneDrive/Desktop/finance/finance_projet_sentiment_analyse/new_data/risk_free_rate_added/BHP_Group_add_risk_free_rate.csv",
    "C:/Users/vongu/OneDrive/Desktop/finance/finance_projet_sentiment_analyse/new_data/risk_free_rate_added/BP_PLC_add_risk_free_rate.csv",
    "C:/Users/vongu/OneDrive/Desktop/finance/finance_projet_sentiment_analyse/new_data/risk_free_rate_added/FMC_Corp_add_risk_free_rate.csv",
    "C:/Users/vongu/OneDrive/Desktop/finance/finance_projet_sentiment_analyse/new_data/risk_free_rate_added/Stora_Enso_add_risk_free_rate.csv",
    "C:/Users/vongu/OneDrive/Desktop/finance/finance_projet_sentiment_analyse/new_data/risk_free_rate_added/Total_Energies_add_risk_free_rate.csv"
]

stock_names = ["BHP_Group", "BP_PLC", "FMC_Corp", "Stora_Enso", "Total_Energies"]

# Call the function
merged_data = merge_put_call_ratios(file_paths, stock_names)

bullish_threshold = -1
bearish_threshold = 1

signal_data = merged_data.copy()
for stock in stock_names:
    signal_data[f"{stock}_Signal"] = calculate_signals_pcr(signal_data[stock], bullish_threshold, bearish_threshold)

weight_data = calculate_dynamic_portfolio_weights(signal_data, stock_names)

weight_data.head(10)

,BHP_Group_Weight,BP_PLC_Weight,FMC_Corp_Weight,Stora_Enso_Weight,Total_Energies_Weight,BHP_Group_Normalized_Weight,BP_PLC_Normalized_Weight,FMC_Corp_Normalized_Weight,Stora_Enso_Normalized_Weight,Total_Energies_Normalized_Weight
Date,,,,,,,,,,
2019-01-03,0.2,0.200000,0.2,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000
2019-01-04,0.2,0.200000,0.2,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000
2019-01-07,0.2,0.180000,0.2,0.180000,0.180000,0.212766,0.191489,0.212766,0.191489,0.191489
2019-01-08,0.2,0.162000,0.2,0.162000,0.162000,0.225734,0.182844,0.225734,0.182844,0.182844
2019-01-09,0.2,0.145800,0.2,0.145800,0.145800,0.238834,0.174110,0.238834,0.174110,0.174110
2019-01-10,0.2,0.131220,0.2,0.131220,0.131220,0.251997,0.165335,0.251997,0.165335,0.165335
2019-01-11,0.2,0.118098,0.2,0.118098,0.118098,0.265149,0.156568,0.265149,0.156568,0.156568
2019-01-14,0.2,0.106288,0.2,0.106288,0.106288,0.278217,0.147856,0.278217,0.147856,0.147856
2019-01-15,0.2,0.095659,0.2,0.095659,0.095659,0.291130,0.139247,0.291130,0.139247,0.139247


In [5]:
import pandas as pd
import os

def merge_put_call_ratios(file_paths, stock_names):
    """
    Merges 'Put-Call Ratio' data for multiple stocks into a single DataFrame aligned by date.
    """
    stock_data = {name: pd.read_csv(file_path) for name, file_path in zip(stock_names, file_paths)}
    put_call_ratios = {name: df[['Date', 'Put-Call Ratio']] for name, df in stock_data.items()}
    merged_data = None

    for name, df in put_call_ratios.items():
        df = df.rename(columns={'Put-Call Ratio': name}) 
        if merged_data is None:  # Initialize with the first DataFrame
            merged_data = df
        else:  # Merge with the existing DataFrame
            merged_data = pd.merge(merged_data, df, on='Date', how='outer')

    merged_data['Date'] = pd.to_datetime(merged_data['Date'])
    merged_data.set_index('Date', inplace=True)

    return merged_data

def calculate_signals_pcr(put_call_ratios, bullish_threshold, bearish_threshold):
    """
    Define function to calculate signals based on Put-Call Ratio, with streak reset for trend changes.
    """
    streak = 0
    signals = []
    previous_signal = None

    for ratio in put_call_ratios:
        if ratio < bullish_threshold:  
            if previous_signal != "Bullish":
                streak = 0  # Reset streak on trend change
            streak += 1
            current_signal = "Bullish"
        elif ratio > bearish_threshold:  
            if previous_signal != "Bearish":
                streak = 0 
            streak -= 1
            current_signal = "Bearish"
        else:  # Neutral
            if previous_signal != "Neutral":
                streak = 0 
            current_signal = "Neutral"

        # Assign Buy, Sell, or Hold based on streak
        if streak >= 3:
            signals.append("Buy")
        elif streak <= -3:
            signals.append("Sell")
        else:
            signals.append("Hold")

        # Update previous signal
        previous_signal = current_signal

    return signals

def calculate_dynamic_portfolio_weights(signal_data, stock_names):
    """
    Calculates daily portfolio weights dynamically based on signal data for multiple stocks.
    """
    # Initialize portfolio weights
    initial_weight = 1 / len(stock_names)

    weight_data = pd.DataFrame(index=signal_data.index)

    # Calculate dynamic weights for each stock
    for stock in stock_names:
        weight_data[f"{stock}_Weight"] = initial_weight  
        for i, signal in enumerate(signal_data[f"{stock}_Signal"]):
            if i > 0:  
                if signal == "Buy":
                    weight_data.iloc[i, weight_data.columns.get_loc(f"{stock}_Weight")] = (
                        weight_data.iloc[i - 1, weight_data.columns.get_loc(f"{stock}_Weight")] * 1.1
                    )  # Increase by 10%
                elif signal == "Sell":
                    weight_data.iloc[i, weight_data.columns.get_loc(f"{stock}_Weight")] = (
                        weight_data.iloc[i - 1, weight_data.columns.get_loc(f"{stock}_Weight")] * 0.9
                    )  # Decrease by 10%
                else:
                    weight_data.iloc[i, weight_data.columns.get_loc(f"{stock}_Weight")] = (
                        weight_data.iloc[i - 1, weight_data.columns.get_loc(f"{stock}_Weight")]
                    )  # Hold

    # Normalize weights to ensure they sum to 1 each day
    weight_data["Total_Weight"] = weight_data[[f"{stock}_Weight" for stock in stock_names]].sum(axis=1)
    for stock in stock_names:
        weight_data[f"{stock}_Normalized_Weight"] = (
            weight_data[f"{stock}_Weight"] / weight_data["Total_Weight"]
        )

    weight_data.drop(columns=["Total_Weight"], inplace=True)

    return weight_data




#a mettre dans un fichier main...
file_paths = [
    "C:/Users/vongu/OneDrive/Desktop/finance/finance_projet_sentiment_analyse/new_data/risk_free_rate_added/BHP_Group_add_risk_free_rate.csv",
    "C:/Users/vongu/OneDrive/Desktop/finance/finance_projet_sentiment_analyse/new_data/risk_free_rate_added/BP_PLC_add_risk_free_rate.csv",
    "C:/Users/vongu/OneDrive/Desktop/finance/finance_projet_sentiment_analyse/new_data/risk_free_rate_added/FMC_Corp_add_risk_free_rate.csv",
    "C:/Users/vongu/OneDrive/Desktop/finance/finance_projet_sentiment_analyse/new_data/risk_free_rate_added/Stora_Enso_add_risk_free_rate.csv",
    "C:/Users/vongu/OneDrive/Desktop/finance/finance_projet_sentiment_analyse/new_data/risk_free_rate_added/Total_Energies_add_risk_free_rate.csv"
]
stock_names = ["BHP_Group", "BP_PLC", "FMC_Corp", "Stora_Enso", "Total_Energies"]
merged_data = merge_put_call_ratios(file_paths, stock_names)

bullish_threshold = -1
bearish_threshold = 1

signal_data = merged_data.copy()
for stock in stock_names:
    signal_data[f"{stock}_Signal"] = calculate_signals_pcr(signal_data[stock], bullish_threshold, bearish_threshold)

weight_data = calculate_dynamic_portfolio_weights(signal_data, stock_names)

print(weight_data)

            BHP_Group_Weight  BP_PLC_Weight  FMC_Corp_Weight  \
Date                                                           
2019-01-03           0.20000   2.000000e-01          0.20000   
2019-01-04           0.20000   2.000000e-01          0.20000   
2019-01-07           0.20000   1.800000e-01          0.20000   
2019-01-08           0.20000   1.620000e-01          0.20000   
2019-01-09           0.20000   1.458000e-01          0.20000   
...                      ...            ...              ...   
2024-12-19           0.00006   1.423995e-30          0.00006   
2024-12-20           0.00006   1.281596e-30          0.00006   
2024-12-23           0.00006   1.281596e-30          0.00006   
2024-12-27           0.00006   1.281596e-30          0.00006   
2024-12-30           0.00006   1.281596e-30          0.00006   

            Stora_Enso_Weight  Total_Energies_Weight  \
Date                                                   
2019-01-03       2.000000e-01           2.000000e-01   